# AI Grader
Use LLM to grade student answer

In [1]:
%pip install langchain openai python-dotenv pandas openpyxl tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dotenv
# Load .env file
dotenv.load_dotenv('.env')

True

In [3]:
deployment_name = "gpt-35-turbo"
model_name = "gpt-35-turbo"
base_folder = "data/ITE3101_practical_tests/ite-3101-practical-test-ab-submissions/"
answer_excel = "student_answer.xlsx"
answer_excel_path = base_folder + answer_excel
standard_answer_path = base_folder + "standard_answer_ab.xlsx"

In [4]:
import pandas as pd

student_answer_df = pd.read_excel(answer_excel_path)
student_answer_df.head()

,Directory,Commit Time,After Deadline,ch01_t01_q1.py Content,ch01_t01_q1.py Commit,ch01_t02_q2.py Content,ch01_t02_q2.py Commit,ch01_t03_q3.py Content,ch01_t03_q3.py Commit,ch01_t04_q4.py Content,ch01_t04_q4.py Commit
0,ite-3101-practical-test-ab-samisop174,2023-11-27 09:03:51,False,def main():\n # Replace pass with your Pyth...,55,import math\ndef main():\n # Replace pass w...,49,def get_number_sequence(n: int) -> str:\n r...,54,"def is_right_angle(a: float, b: float, c: floa...",12
1,ite-3101-practical-test-ab-LCK312,2023-11-27 09:02:33,False,def main():\n # Replace pass with your Pyth...,72,def main():\n # Replace pass with your Pyth...,43,def get_number_sequence(n: int) -> str:\n r...,7,"def is_right_angle(a: float, b: float, c: floa...",18
2,ite-3101-practical-test-ab-rivenkey,2023-11-27 09:01:14,False,def main():\n while True:\n try:\n ...,44,def main():\n while True:\n num1 = in...,73,def get_number_sequence(n: int) -> str:\n r...,26,"def is_right_angle(a: float, b: float, c: floa...",14
3,ite-3101-practical-test-ab-VincentWun,2023-11-27 09:00:09,False,def main():\n # Replace pass with your Pyth...,23,def main():\n # Replace pass with your Pyth...,37,def get_number_sequence(n: int) -> str:\n r...,40,"def is_right_angle(a: float, b: float, c: floa...",48
4,ite-3101-practical-test-ab-JacksonChan762,2023-11-27 09:02:57,False,def main():\n while True:\n try:\n ...,89,import math\ndef main():\n # Replace pass w...,27,def get_number_sequence(n: int) -> str:\n r...,45,"def is_right_angle(a: float, b: float, c: floa...",35


In [5]:
standard_answer = pd.read_excel(standard_answer_path)
standard_answer.head()

,Question Name,Instruction,Starter,Answer,Mark
0,ch01_t01_q1.py,Write a Python program that will ask the user ...,def main():\n # Replace pass with your Pyth...,def main():\n while True:\n try:\n ...,20
1,ch01_t02_q2.py,The area of a trapezoid is given by the follow...,def main():\n # Replace pass with your Pyth...,def main():\n try:\n upper = float(i...,20
2,ch01_t03_q3.py,(a)\tWrite a method “get_number_sequence” to g...,def get_number_sequence(n: int) -> str:\n r...,def get_number_sequence(n: int) -> str:\n r...,20
3,ch01_t04_q4.py,"Complete the following Python program, which i...","def is_right_angle(a: float, b: float, c: floa...","def is_right_angle(a: float, b: float, c: floa...",20


In [6]:
standard_answer_dict = standard_answer.set_index(
    'Question Name').to_dict(orient='index')
standard_answer_dict

{'ch01_t01_q1.py': {'Instruction': "Write a Python program that will ask the user to enter an integer, read the user's response, and tell the user whether the number entered is evenly divisible by both 3 and 5.  You can assume the user will always provide valid input(s) so you do not have to deal with input validation.",
  'Starter': 'def main():\n    # Replace pass with your Python code.\n    pass\n\n\nif __name__ == "__main__":\n    # execute only if run as a script\n    main()',
  'Answer': 'def main():\n    while True:\n        try:\n            num = int(input("Type an integer:"))\n            break\n        except ValueError:\n            print(\'Please enter an integer.\')\n\n    if num % 3 == 0 and num % 5 == 0:\n        print(f\'{num} is divisible by both 3 and 5.\')\n    else:\n        print(f\'{num} is not divisible by both 3 and 5.\')\n\n\nif __name__ == "__main__":\n    # execute only if run as a script\n    main()',
  'Mark': 20},
 'ch01_t02_q2.py': {'Instruction': 'The a

In [7]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
import json
import langchain
langchain.debug = False


llm = AzureChatOpenAI(
    deployment_name=deployment_name,
    model_name=model_name,
    temperature=0.1
)


def score_answer(instruction, starter, answer, mark, student_answer, student_commit):
    template = "You are a Python programming instructor who grades student Python exercises."
    # load grader_prompt.txt
    with open("grader_prompt.txt") as f:
        grader_prompt = f.read()

    chat_prompt = ChatPromptTemplate.from_messages([
        ("system", template),
        ("human", grader_prompt),
    ])
    # print(chat_prompt)
    chain = chat_prompt | llm
    data = {"instruction": instruction,
            "starter": starter,
            "answer": answer,
            "mark": mark,
            "student_answer": student_answer,
            "student_commit": student_commit}

    r = chain.invoke(data).content
    return r

/home/codespace/.local/lib/python3.8/site-packages/langchain/chat_models/azure_openai.py:156: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://eastus.api.cognitive.microsoft.com/ to https://eastus.api.cognitive.microsoft.com/openai.
  warnings.warn(
/home/codespace/.local/lib/python3.8/site-packages/langchain/chat_models/azure_openai.py:163: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/home/codespace/.local/lib/python3.8/site-packages/langchain/chat_models/azure_openai.py:171: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/ope

In [10]:
import json
from tqdm import tqdm

json_decode_error_count = 0
type_error_count = 0
rows = []
for index, row in tqdm(student_answer_df.iterrows(), total=len(student_answer_df)):
    for key, value in standard_answer_dict.items():
        question = key
        instruction = value["Instruction"]
        starter = value["Starter"]
        answer = value["Answer"]
        mark = value["Mark"]
        student_answer = row[key + " Content"]
        student_commit = row[key + " Commit"]

        for _ in range(3):  # Retry 3 times
            try:
                result = score_answer(
                    instruction, starter, answer, mark, student_answer, student_commit)
                result = json.loads(result)  # Parse the JSON response
                row[key + " Score"] = result["score"]
                row[key + " Comments"] = result["comments"]
                row[key + " Calculation"] = result["calculation"]      
                break  # Break the loop if successful
            except json.JSONDecodeError:     
                json_decode_error_count += 1     
                continue  # Retry if JSON decoding error occurs
            except TypeError:
                type_error_count += 1
                continue

    rows.append(row)

print(f"Total JSONDecodeError: {json_decode_error_count}")
print(f"Total TypeError: {type_error_count}")

100%|██████████| 53/53 [08:57<00:00, 10.15s/it]

Total JSONDecodeError: 10
Total TypeError: 0


In [13]:
scored_df = pd.DataFrame(rows)
scored_df.head()

,Directory,Commit Time,After Deadline,ch01_t01_q1.py Content,ch01_t01_q1.py Commit,ch01_t02_q2.py Content,ch01_t02_q2.py Commit,ch01_t03_q3.py Content,ch01_t03_q3.py Commit,ch01_t04_q4.py Content,...,ch01_t01_q1.py Calculation,ch01_t02_q2.py Score,ch01_t02_q2.py Comments,ch01_t02_q2.py Calculation,ch01_t03_q3.py Score,ch01_t03_q3.py Comments,ch01_t03_q3.py Calculation,ch01_t04_q4.py Score,ch01_t04_q4.py Comments,ch01_t04_q4.py Calculation
0,ite-3101-practical-test-ab-samisop174,2023-11-27 09:03:51,False,def main():\n # Replace pass with your Pyth...,55,import math\ndef main():\n # Replace pass w...,49,def get_number_sequence(n: int) -> str:\n r...,54,"def is_right_angle(a: float, b: float, c: floa...",...,The student's code is the same as the standard...,20,Great job! Your code is correct and follows th...,The student's code is the same as the standard...,20,Great job! Your code is correct and follows th...,All lines of code in the StudentAnswer are cor...,10,The student has attempted the question but the...,10/20 = 0.5
1,ite-3101-practical-test-ab-LCK312,2023-11-27 09:02:33,False,def main():\n # Replace pass with your Pyth...,72,def main():\n # Replace pass with your Pyth...,43,def get_number_sequence(n: int) -> str:\n r...,7,"def is_right_angle(a: float, b: float, c: floa...",...,The student's code is the same as the standard...,18,The code works correctly and produces the expe...,The StudentAnswer and StandardAnswer are mostl...,20,Great job! Your code works perfectly and meets...,All lines of code are correct and the logic is...,14,The code is almost correct. The is_right_angle...,is_right_angle function: 6/6 lines correct. ma...
2,ite-3101-practical-test-ab-rivenkey,2023-11-27 09:01:14,False,def main():\n while True:\n try:\n ...,44,def main():\n while True:\n num1 = in...,73,def get_number_sequence(n: int) -> str:\n r...,26,"def is_right_angle(a: float, b: float, c: floa...",...,All lines of code in the StudentAnswer are the...,16,The code has some syntax errors. The input and...,16 = 1 (input) + 1 (input) + 1 (input) + 1 (co...,10,The code has some syntax errors and does not f...,10 = 1 (correct get_number_sequence function) ...,10,The student has attempted the question but the...,10/20 = 0.5
3,ite-3101-practical-test-ab-VincentWun,2023-11-27 09:00:09,False,def main():\n # Replace pass with your Pyth...,23,def main():\n # Replace pass with your Pyth...,37,def get_number_sequence(n: int) -> str:\n r...,40,"def is_right_angle(a: float, b: float, c: floa...",...,Total lines: 23\nCorrect lines: 15\nIncorrect ...,16,The code is mostly correct. The program accept...,The StudentAnswer correctly accepts user input...,18,The code is mostly correct. The get_number_seq...,get_number_sequence function: 4 points (correc...,16,"The code logic is correct. However, the Studen...",16 = 1 (is_right_angle function) + 1 (if state...
4,ite-3101-practical-test-ab-JacksonChan762,2023-11-27 09:02:57,False,def main():\n while True:\n try:\n ...,89,import math\ndef main():\n # Replace pass w...,27,def get_number_sequence(n: int) -> str:\n r...,45,"def is_right_angle(a: float, b: float, c: floa...",...,Total lines: 23. Correct lines: 15. score = 15...,18,The code works correctly and produces the expe...,The StudentAnswer is almost identical to the S...,20,Great job! Your code is correct and follows th...,All lines of code in the StudentAnswer are cor...,10,The code logic is partially correct. The input...,10/20 = 0.5


In [14]:
default_value = 0  # or any other value you want to replace NaN with
scored_df[scored_df.filter(like='Score').columns] = scored_df.filter(like='Score').fillna(default_value).astype(int)
scored_df[scored_df.filter(like='Commit').columns] = scored_df.filter(like='Commit').fillna(default_value).astype(int)
scored_df['total_score'] = scored_df.filter(like='Score').sum(axis=1)
scored_df.head()

,Directory,Commit Time,After Deadline,ch01_t01_q1.py Content,ch01_t01_q1.py Commit,ch01_t02_q2.py Content,ch01_t02_q2.py Commit,ch01_t03_q3.py Content,ch01_t03_q3.py Commit,ch01_t04_q4.py Content,...,ch01_t02_q2.py Score,ch01_t02_q2.py Comments,ch01_t02_q2.py Calculation,ch01_t03_q3.py Score,ch01_t03_q3.py Comments,ch01_t03_q3.py Calculation,ch01_t04_q4.py Score,ch01_t04_q4.py Comments,ch01_t04_q4.py Calculation,total_score
0,ite-3101-practical-test-ab-samisop174,1701075831000000000,False,def main():\n # Replace pass with your Pyth...,55,import math\ndef main():\n # Replace pass w...,49,def get_number_sequence(n: int) -> str:\n r...,54,"def is_right_angle(a: float, b: float, c: floa...",...,20,Great job! Your code is correct and follows th...,The student's code is the same as the standard...,20,Great job! Your code is correct and follows th...,All lines of code in the StudentAnswer are cor...,10,The student has attempted the question but the...,10/20 = 0.5,70
1,ite-3101-practical-test-ab-LCK312,1701075753000000000,False,def main():\n # Replace pass with your Pyth...,72,def main():\n # Replace pass with your Pyth...,43,def get_number_sequence(n: int) -> str:\n r...,7,"def is_right_angle(a: float, b: float, c: floa...",...,18,The code works correctly and produces the expe...,The StudentAnswer and StandardAnswer are mostl...,20,Great job! Your code works perfectly and meets...,All lines of code are correct and the logic is...,14,The code is almost correct. The is_right_angle...,is_right_angle function: 6/6 lines correct. ma...,72
2,ite-3101-practical-test-ab-rivenkey,1701075674000000000,False,def main():\n while True:\n try:\n ...,44,def main():\n while True:\n num1 = in...,73,def get_number_sequence(n: int) -> str:\n r...,26,"def is_right_angle(a: float, b: float, c: floa...",...,16,The code has some syntax errors. The input and...,16 = 1 (input) + 1 (input) + 1 (input) + 1 (co...,10,The code has some syntax errors and does not f...,10 = 1 (correct get_number_sequence function) ...,10,The student has attempted the question but the...,10/20 = 0.5,56
3,ite-3101-practical-test-ab-VincentWun,1701075609000000000,False,def main():\n # Replace pass with your Pyth...,23,def main():\n # Replace pass with your Pyth...,37,def get_number_sequence(n: int) -> str:\n r...,40,"def is_right_angle(a: float, b: float, c: floa...",...,16,The code is mostly correct. The program accept...,The StudentAnswer correctly accepts user input...,18,The code is mostly correct. The get_number_seq...,get_number_sequence function: 4 points (correc...,16,"The code logic is correct. However, the Studen...",16 = 1 (is_right_angle function) + 1 (if state...,65
4,ite-3101-practical-test-ab-JacksonChan762,1701075777000000000,False,def main():\n while True:\n try:\n ...,89,import math\ndef main():\n # Replace pass w...,27,def get_number_sequence(n: int) -> str:\n r...,45,"def is_right_angle(a: float, b: float, c: floa...",...,18,The code works correctly and produces the expe...,The StudentAnswer is almost identical to the S...,20,Great job! Your code is correct and follows th...,All lines of code in the StudentAnswer are cor...,10,The code logic is partially correct. The input...,10/20 = 0.5,63


In [15]:
score_columns = scored_df.filter(like='Score').columns
scored_df = scored_df[[
    col for col in scored_df.columns if col not in score_columns] + list(score_columns)]
scored_df.head()

,Directory,Commit Time,After Deadline,ch01_t01_q1.py Content,ch01_t01_q1.py Commit,ch01_t02_q2.py Content,ch01_t02_q2.py Commit,ch01_t03_q3.py Content,ch01_t03_q3.py Commit,ch01_t04_q4.py Content,...,ch01_t02_q2.py Calculation,ch01_t03_q3.py Comments,ch01_t03_q3.py Calculation,ch01_t04_q4.py Comments,ch01_t04_q4.py Calculation,total_score,ch01_t01_q1.py Score,ch01_t02_q2.py Score,ch01_t03_q3.py Score,ch01_t04_q4.py Score
0,ite-3101-practical-test-ab-samisop174,1701075831000000000,False,def main():\n # Replace pass with your Pyth...,55,import math\ndef main():\n # Replace pass w...,49,def get_number_sequence(n: int) -> str:\n r...,54,"def is_right_angle(a: float, b: float, c: floa...",...,The student's code is the same as the standard...,Great job! Your code is correct and follows th...,All lines of code in the StudentAnswer are cor...,The student has attempted the question but the...,10/20 = 0.5,70,20,20,20,10
1,ite-3101-practical-test-ab-LCK312,1701075753000000000,False,def main():\n # Replace pass with your Pyth...,72,def main():\n # Replace pass with your Pyth...,43,def get_number_sequence(n: int) -> str:\n r...,7,"def is_right_angle(a: float, b: float, c: floa...",...,The StudentAnswer and StandardAnswer are mostl...,Great job! Your code works perfectly and meets...,All lines of code are correct and the logic is...,The code is almost correct. The is_right_angle...,is_right_angle function: 6/6 lines correct. ma...,72,20,18,20,14
2,ite-3101-practical-test-ab-rivenkey,1701075674000000000,False,def main():\n while True:\n try:\n ...,44,def main():\n while True:\n num1 = in...,73,def get_number_sequence(n: int) -> str:\n r...,26,"def is_right_angle(a: float, b: float, c: floa...",...,16 = 1 (input) + 1 (input) + 1 (input) + 1 (co...,The code has some syntax errors and does not f...,10 = 1 (correct get_number_sequence function) ...,The student has attempted the question but the...,10/20 = 0.5,56,20,16,10,10
3,ite-3101-practical-test-ab-VincentWun,1701075609000000000,False,def main():\n # Replace pass with your Pyth...,23,def main():\n # Replace pass with your Pyth...,37,def get_number_sequence(n: int) -> str:\n r...,40,"def is_right_angle(a: float, b: float, c: floa...",...,The StudentAnswer correctly accepts user input...,The code is mostly correct. The get_number_seq...,get_number_sequence function: 4 points (correc...,"The code logic is correct. However, the Studen...",16 = 1 (is_right_angle function) + 1 (if state...,65,15,16,18,16
4,ite-3101-practical-test-ab-JacksonChan762,1701075777000000000,False,def main():\n while True:\n try:\n ...,89,import math\ndef main():\n # Replace pass w...,27,def get_number_sequence(n: int) -> str:\n r...,45,"def is_right_angle(a: float, b: float, c: floa...",...,The StudentAnswer is almost identical to the S...,Great job! Your code is correct and follows th...,All lines of code in the StudentAnswer are cor...,The code logic is partially correct. The input...,10/20 = 0.5,63,15,18,20,10


In [17]:
import os
excel_file_path = os.path.join(base_folder, 'student_score.xlsx')
scored_df.to_excel(excel_file_path, index=False)